In [10]:
import pandas as pd
uselog = pd.read_csv('use_log.csv')  # データを読み込み
uselog.isnull().sum()  # 欠損値の数を出力

log_id         0
customer_id    0
usedate        0
dtype: int64

In [8]:
uselog.dtypes  # データタイプを確認

log_id                 object
customer_id            object
usedate        datetime64[ns]
年月                     object
dtype: object

In [13]:
uselog["usedate"]=pd.to_datetime(uselog["usedate"])  # データをdatetime型に変換
uselog["年月"] = uselog["usedate"].dt.strftime("%Y%m")  # 新たな列“年月”を年月単位で文字列として作成
uselog[["usedate", "年月"]].head()

,usedate,年月
0,2018-04-01,201804
1,2018-04-01,201804
2,2018-04-01,201804
3,2018-04-01,201804
4,2018-04-01,201804


In [2]:
customer = pd.read_csv('customer_join.csv')
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

In [25]:
uselog_months = uselog.groupby(["年月","customer_id"], as_index=False).count()  # 年月、顧客毎に集計
uselog_months.rename(columns = {"log_id":"count"}, inplace=True)
del uselog_months["usedate"]  # 余分なusedateは削除
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [18]:
year_months = list(uselog_months["年月"].unique())  # 年月データをリストに格納
print(year_months)

['201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903']


In [31]:
valid = pd.pivot_table(uselog_months,index='年月',columns='customer_id',values='count')
valid

customer_id,AS002855,AS008805,AS009013,AS009373,AS015233,AS015315,AS015739,AS015746,AS019120,AS019860,...,TS989364,TS989482,TS993439,TS995238,TS995299,TS995853,TS998593,TS999079,TS999231,TS999855
年月,,,,,,,,,,,,,,,,,,,,,
201804,4.0,NaN,2.0,3.0,NaN,6.0,7.0,4.0,NaN,6.0,...,3.0,6.0,NaN,NaN,4.0,NaN,NaN,7.0,6.0,6.0
201805,5.0,NaN,NaN,4.0,7.0,3.0,5.0,1.0,NaN,8.0,...,8.0,7.0,10.0,NaN,7.0,NaN,NaN,5.0,4.0,3.0
201806,5.0,8.0,NaN,4.0,7.0,3.0,6.0,2.0,NaN,6.0,...,8.0,4.0,3.0,NaN,2.0,NaN,NaN,6.0,1.0,6.0
201807,5.0,7.0,NaN,7.0,5.0,6.0,8.0,3.0,NaN,4.0,...,7.0,8.0,NaN,NaN,5.0,NaN,NaN,2.0,5.0,4.0
201808,3.0,5.0,NaN,6.0,11.0,3.0,5.0,3.0,8.0,7.0,...,7.0,6.0,NaN,NaN,7.0,NaN,NaN,3.0,3.0,3.0
201809,7.0,2.0,NaN,6.0,9.0,7.0,6.0,NaN,5.0,5.0,...,9.0,7.0,NaN,NaN,5.0,NaN,9.0,4.0,4.0,5.0
201810,3.0,2.0,NaN,5.0,7.0,4.0,5.0,NaN,3.0,7.0,...,9.0,2.0,NaN,NaN,4.0,NaN,9.0,6.0,5.0,4.0
201811,5.0,3.0,NaN,4.0,7.0,7.0,4.0,NaN,NaN,7.0,...,10.0,4.0,NaN,NaN,5.0,NaN,9.0,6.0,5.0,4.0
201812,2.0,5.0,NaN,4.0,9.0,5.0,4.0,NaN,NaN,5.0,...,6.0,3.0,NaN,8.0,4.0,NaN,7.0,9.0,8.0,5.0


In [27]:
predict_data = pd.DataFrame()
for i in range(6, len(year_months)):  # 2018年10月～2019年03月
    tmp = uselog_months.loc[uselog_months["年月"] == year_months[i]]  # 年月=来月（予測対象月）
    tmp.rename(columns = {"count":"count_pred"}, inplace=True)  # count_pred=来月の利用回数（正解）
    for j in range(1, 7):
        tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-j]]
        del tmp_before["年月"]
        tmp_before.rename(columns={"count":"count_{}".format(j-1)}, inplace=True)  # count_0:今月、count_1:先月
        tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    predict_data = pd.concat([predict_data, tmp], ignore_index=True)
predict_data.head()

,年月,customer_id,count_pred,count_0,count_1,count_2,count_3,count_4,count_5
0,201810,AS002855,3,7.0,3.0,5.0,5.0,5.0,4.0
1,201810,AS008805,2,2.0,5.0,7.0,8.0,NaN,NaN
2,201810,AS009373,5,6.0,6.0,7.0,4.0,4.0,3.0
3,201810,AS015233,7,9.0,11.0,5.0,7.0,7.0,NaN
4,201810,AS015315,4,7.0,3.0,6.0,3.0,3.0,6.0


In [33]:
predict_data = predict_data.dropna()  # 欠損値除去（対象を6ヶ月以上滞在している顧客に絞る）
predict_data = predict_data.reset_index(drop=True)  # indexを初期化
predict_data = pd.merge(predict_data, customer[["customer_id","start_date"]], on="customer_id", how="left")  # start_date列を追加
predict_data.head()

,年月,customer_id,count_pred,count_0,count_1,count_2,count_3,count_4,count_5,start_date
0,201810,AS002855,3,7.0,3.0,5.0,5.0,5.0,4.0,2016-11-01
1,201810,AS009373,5,6.0,6.0,7.0,4.0,4.0,3.0,2015-11-01
2,201810,AS015315,4,7.0,3.0,6.0,3.0,3.0,6.0,2015-07-01
3,201810,AS015739,5,6.0,5.0,8.0,6.0,5.0,7.0,2017-06-01
4,201810,AS019860,7,5.0,7.0,4.0,6.0,8.0,6.0,2017-10-01


In [39]:
import sys
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
from dateutil.relativedelta import relativedelta  # 日付の比較
predict_data["period"] = None
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = delta.years*12 + delta.months  # 会員期間を算出
predict_data.head()

C:\Users\rytmy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,年月,customer_id,count_pred,count_0,count_1,count_2,count_3,count_4,count_5,start_date,now_date,period
0,201810,AS002855,3,7.0,3.0,5.0,5.0,5.0,4.0,2016-11-01,2018-10-01,23
1,201810,AS009373,5,6.0,6.0,7.0,4.0,4.0,3.0,2015-11-01,2018-10-01,35
2,201810,AS015315,4,7.0,3.0,6.0,3.0,3.0,6.0,2015-07-01,2018-10-01,39
3,201810,AS015739,5,6.0,5.0,8.0,6.0,5.0,7.0,2017-06-01,2018-10-01,16
4,201810,AS019860,7,5.0,7.0,4.0,6.0,8.0,6.0,2017-10-01,2018-10-01,12


In [49]:
predict_data = predict_data.loc[predict_data["start_date"]>=pd.to_datetime("20180401")]  # 2018年4月以降に入会した顧客を対象
predict_data.to_csv("kadai.csv")

In [41]:
from sklearn import linear_model
model = linear_model.LinearRegression()
X = predict_data[["count_0","count_1","count_2","count_3","count_4","count_5","period"]]
y = predict_data["count_pred"]

In [42]:
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [47]:
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.6161452631955938
0.578522659830823


In [48]:
coef = pd.DataFrame({"feature_names":X.columns, "coefficient":model.coef_})
coef

,feature_names,coefficient
0,count_0,0.343125
1,count_1,0.192777
2,count_2,0.174675
3,count_3,0.181507
4,count_4,0.078595
5,count_5,0.091675
6,period,0.083402
